In [1]:
import pandas as pd
import numpy as np
import json
import ast
import collections
import re
import pprint
import csv
from tqdm import tqdm
import pymongo


In [2]:
file = "Resources/movies_metadata.csv"
df = pd.read_csv(file)
print("Start Shape: " + str(df.shape))
pd.set_option('display.max_columns', None)
df.fillna(-9999999, inplace=True)
df.head()
# df.to_csv('Resources/movies_metadata_pandas.csv')
# # df["id"] = df["id"].apply(pd.to_numeric, errors="coerce")
df["id"] = pd.to_numeric(df["id"].replace([np.inf, -np.inf], np.nan).dropna(), downcast="integer", errors="coerce")
df = df.dropna()
df["id"] = df["id"].astype(int)
# df.head()
print(df.dtypes)
print("End Shape: " + str(df.shape))

Start Shape: (45466, 24)
adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                         int32
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object
End Shape: (45463, 24)


In [3]:
# Subroutine that does the following things:
# Grabs the JSON text from the column key spcified and returns an Object
# Assumes there is a list of objects and iterates over them
# Grab the data for the key and append it to a local list
# Once we are done iterating over the object, replace the element that
# we started with with a colon seperated list on the key words
def getJSONDictElements(df, colKey, dictKey, seperator, totalDict):
    with tqdm(total=df.shape[0]) as pbar:
        for index, row in df.iterrows():
            pbar.update(1)
            json_string = row[colKey]
            if not isinstance(json_string, str):
                # skip lines that aren't strings
                continue
            if "{" not in json_string:
                # skip lines that aren't dict's
                continue
            obj = ast.literal_eval(json_string)
            elementList = []
            for item in obj:
            # pprint.pprint(item)
                if len(item[dictKey]) > 0:
                    elementList.append(item[dictKey])
                    if item[dictKey] not in totalDict:
                        totalDict[item[dictKey]] = 1
                    else:
                        totalDict[item[dictKey]] += 1
            df.at[index, colKey] = seperator.join(elementList)

In [4]:
file = "Resources/genre_list.csv"
total_genres = {}
getJSONDictElements(df, "genres", "name", ":", total_genres)
with open(file, 'w', newline='') as csvFile:
    csvwriter = csv.writer(csvFile)
    for key in sorted(total_genres.keys()):
        if total_genres[key] < 5:
            # skip the production company names
            continue
        csvwriter.writerow([key, total_genres[key]])

df.head()

100%|██████████| 45463/45463 [00:07<00:00, 5756.50it/s]


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation:Comedy:Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,-9999999,Toy Story,False,7.7,5415.0
1,False,-9999999,65000000,Adventure:Fantasy:Family,-9999999,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance:Comedy,-9999999,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,-9999999,16000000,Comedy:Drama:Romance,-9999999,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,-9999999,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
file = "Resources/keywords.csv"
df_key = pd.read_csv(file)
df_key.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [6]:
total_key_words = {}
getJSONDictElements(df_key, "keywords", "name", ":", total_key_words)
df_key.head()

100%|██████████| 46419/46419 [00:08<00:00, 5687.26it/s]


,id,keywords
0,862,jealousy:toy:boy:friendship:friends:rivalry:bo...
1,8844,board game:disappearance:based on children's b...
2,15602,fishing:best friend:duringcreditsstinger:old men
3,31357,based on novel:interracial relationship:single...
4,11862,baby:midlife crisis:confidence:aging:daughter:...


In [7]:
print(df["id"].dtypes)
print(df_key.dtypes)
joined_df = pd.merge(left=df, right=df_key, left_on="id", right_on="id")
print(joined_df.head())
print("End Shape: " + str(joined_df.shape))

int32
id           int64
keywords    object
dtype: object
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                           -9999999  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                           -9999999  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                     genres                              homepage     id  \
0   Animation:Comedy:Family  http://toystory.disney.com/toy-story    862   
1  Adventure:Fantasy:Family                              -9999999   8844   
2            Romance:Comedy                              -9999999  15602   
3      Comedy:Drama:Romance                              -9999999  31357   
4                    Comedy                              -9999999  11862   

     imdb_id original_language               o

In [8]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Declare the database
db = client.movies_db
collection = db['movie_table']
collection.drop() # drop the table to start fresh
collection = db['movie_table']

# Convert the joined_df to a dict
data_dict = joined_df.to_dict("records")

collection.insert_many(data_dict)

In [9]:
joined_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,Animation:Comedy:Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,-9999999,Toy Story,False,7.7,5415.0,jealousy:toy:boy:friendship:friends:rivalry:bo...
1,False,-9999999,65000000,Adventure:Fantasy:Family,-9999999,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,board game:disappearance:based on children's b...
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,Romance:Comedy,-9999999,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,fishing:best friend:duringcreditsstinger:old men
3,False,-9999999,16000000,Comedy:Drama:Romance,-9999999,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,based on novel:interracial relationship:single...
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,Comedy,-9999999,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,baby:midlife crisis:confidence:aging:daughter:...


In [12]:
print("posts: " + str(db.list_collection_names ) )

posts: <bound method Database.list_collection_names of Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'movies_db')>
